In [8]:
import sys
import os

# Add the src directory to the Python path
src_path = os.path.abspath('./src')
if src_path not in sys.path:
    sys.path.append(src_path)
print("Current working directory:", os.getcwd())

Current working directory: z:\AST-With-TB-Classify


In [ ]:
import os 
import sys
import torch
from models.ast_models import ASTModel
os.environ['TORCH_HOME'] = '../pretrained_models'  

# Define the AST-P model architecture ทำตาม Paper 
achitecture = ASTModel(
            label_dim=2,             # จำนวนคลาส (PTB, NON-PTB)
            fstride=10,              # ต้องเป็น 10 สำหรับ AudioSet pretrain
            tstride=10,              # ต้องเป็น 10 สำหรับ AudioSet pretrain
            input_fdim=128,          # ค่ามาตรฐาน
            input_tdim=100,          # ปรับตามความยาว spectrogram ของคุณ 
            imagenet_pretrain=True,  # เปิดใช้งาน
            audioset_pretrain=True,  # เปิดใช้งาน (AST-P)
            model_size='base384'     # ต้องเป็น base384
        )
ast_mdl = achitecture

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=108


In [27]:
import os
import json
import random
from collections import defaultdict

def prepare_ast_patient_split_clean(root_path, train_ratio=0.8):
    # 1. จัดกลุ่มไฟล์ตามรหัสผู้ป่วย (Patient ID: 00x)
    patient_groups = {
        "Cough_PTB": defaultdict(list),
        "Cough_Non-PTB": defaultdict(list)
    }
    
    class_map = {"Cough_PTB": "1", "Cough_Non-PTB": "0"}

    for folder_name, label_idx in class_map.items():
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.exists(folder_path): continue
            
        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                # ดึง Patient ID (00x) จากส่วนหน้าสุดของชื่อไฟล์
                patient_id = file.split('_')[0] 
                full_path = os.path.abspath(os.path.join(folder_path, file))
                patient_groups[folder_name][patient_id].append(full_path)

    train_list, eval_list = [], []
    stats = {
        "train": {"patients": [], "ptb_samples": 0, "non_ptb_samples": 0},
        "eval": {"patients": [], "ptb_samples": 0, "non_ptb_samples": 0}
    }

    # 2. แบ่งข้อมูลรายกลุ่มผู้ป่วย (Patient Level) แยกตามคลาส
    for folder_name, groups in patient_groups.items():
        patient_ids = list(groups.keys())
        random.seed(42) # ล็อก Seed เพื่อให้ผลลัพธ์คงที่
        random.shuffle(patient_ids)
        
        split_idx = int(len(patient_ids) * train_ratio)
        train_ids = patient_ids[:split_idx]
        eval_ids = patient_ids[split_idx:]

        # กลุ่ม Train
        for p_id in train_ids:
            stats["train"]["patients"].append(p_id) # เก็บเฉพาะรหัสผู้ป่วย
            for path in groups[p_id]:
                train_list.append({"wav": path, "labels": class_map[folder_name]})
                if folder_name == "Cough_PTB": stats["train"]["ptb_samples"] += 1
                else: stats["train"]["non_ptb_samples"] += 1
        
        # กลุ่ม Eval
        for p_id in eval_ids:
            stats["eval"]["patients"].append(p_id) # เก็บเฉพาะรหัสผู้ป่วย
            for path in groups[p_id]:
                eval_list.append({"wav": path, "labels": class_map[folder_name]})
                if folder_name == "Cough_PTB": stats["eval"]["ptb_samples"] += 1
                else: stats["eval"]["non_ptb_samples"] += 1

    # 3. บันทึกไฟล์ JSON
    with open('train_data.json', 'w') as f:
        json.dump({"data": train_list}, f, indent=4)
    with open('eval_data.json', 'w') as f:
        json.dump({"data": eval_list}, f, indent=4)

    # 4. แสดงผลสรุปที่สะอาดขึ้น
    print("="*50)
    print("AST DATA PREPARATION SUMMARY")
    print("="*50)
    
    print(f"\n[TRAIN SET]")
    print(f"Total Unique Patients: {len(stats['train']['patients'])} คน")
    print(f"Patient IDs: {', '.join(sorted(stats['train']['patients']))}")
    print(f"Samples: PTB (Class 1) = {stats['train']['ptb_samples']} | Non-PTB (Class 0) = {stats['train']['non_ptb_samples']}")
    
    print(f"\n[EVAL SET]")
    print(f"Total Unique Patients: {len(stats['eval']['patients'])} คน")
    print(f"Patient IDs: {', '.join(sorted(stats['eval']['patients']))}")
    print(f"Samples: PTB (Class 1) = {stats['eval']['ptb_samples']} | Non-PTB (Class 0) = {stats['eval']['non_ptb_samples']}")
    print("="*50)

# เรียกใช้งาน
prepare_ast_patient_split_clean("./Data")

AST DATA PREPARATION SUMMARY

[TRAIN SET]
Total Unique Patients: 11 คน
Patient IDs: 003, 004, 005, 007, 008, 009, 011, 012, 013, 014, 016
Samples: PTB (Class 1) = 98 | Non-PTB (Class 0) = 176

[EVAL SET]
Total Unique Patients: 4 คน
Patient IDs: 001, 002, 006, 015
Samples: PTB (Class 1) = 89 | Non-PTB (Class 0) = 149
